In [1]:
import tensorflow as tf
import os
import numpy as np
import sys

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
def _img_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value.flatten()))

def process_img(filename, size=None, crop=True):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    
    try:
        image_shape = tf.image.extract_jpeg_shape(image_string)
        h = image_shape[0]
        w = image_shape[1]
        s = tf.minimum(h, w)
        image_decoded = tf.image.resize_image_with_crop_or_pad(image_decoded, s, s)
    except:
        pass

    image_resized = tf.image.resize_images(image_decoded, size) if size is not None else image_decoded
    return tf.to_float(image_resized)

def make_example(imgs):
    imgs = tf.train.FeatureList(feature=[_img_feature(img) for img in imgs])
    feature_lists = tf.train.FeatureLists(feature_list={'imgs':imgs})
    return tf.train.SequenceExample(feature_lists=feature_lists)

def make_one_example(img):
    feature={'img':_img_feature(img)}
    return tf.train.Example(features=tf.train.Features(feature=feature))
    
def create_pipeline(sess, params):
    files = tf.data.Dataset.list_files(params.train_path)
    files = files.take(params.total_train_sample)
    files = files.map(lambda x: process_img(x, params.input_shape[0:2], crop=True), num_parallel_calls=7)
    files = files.batch(params.batch_size)
    return files.make_initializable_iterator()

def write():
    class Params:
        def __init__(self):
            # aws
            self.train_path = '/home/ubuntu/images/data/unlabeled2017/*.jpg'
            # mbp
            #self.train_path = '/Users/paul/Work/ai/images/val2017/*.jpg'
            # work
            # self.train_path = 'data/train/val2017/*.jpg'
            self.out_path = '/home/ubuntu/work/images/train'
            self.total_train_sample = 1000*4*30
            self.batch_size = 4
            self.input_shape = [256,256]

    params = Params()
            
    tf.reset_default_graph()
    sess = tf.InteractiveSession()

    iterator = create_pipeline(sess, params)
    next_batch = iterator.get_next()

    sess.run(iterator.initializer)
    b = 0
    i = 0
    f = 0
    writer = None
    print('Starting\n')
    while True:
        try:
            batch = sess.run(next_batch)

            if writer is None:
                output_file = os.path.join(params.out_path, str(f) + ".tfr")
                writer = tf.python_io.TFRecordWriter(output_file)

            for img in batch:
                writer.write(make_one_example(img).SerializeToString())

            b += 1
            i += 1
            if b%1000 == 0:
                f += 1
                b=0
                writer.close()
                writer = None

            sys.stdout.write("\r")
            sys.stdout.write("Done %i/%i" % (i, params.total_train_sample/params.batch_size))
        except tf.errors.InvalidArgumentError:
            continue
        except tf.errors.OutOfRangeError:
            break
    
    if writer is not None:
        writer.close()

    print("\nDone")

write()

Starting

Done 29763/30000

In [ ]:
class TrainParams:
     def __init__(self):
        self.train_path = 'data/train/val/*.tfr'
        self.total_train_sample = 8
        self.batch_size = 4
        self.input_shape = [256,256]

def process_tf(x, shape=None):
    parsed_features = tf.parse_single_example(x, features={
        'img':tf.FixedLenFeature([shape[0]*shape[1]*3], dtype=tf.float32)
    })
    imgs = parsed_features['img']
    imgs = tf.reshape(imgs, shape + [3])
    return imgs
        
def create_tf_pipeline(sess, params):
    filenames = tf.data.Dataset.list_files(params.train_path).take(params.total_train_sample)
    filenames_iterator = filenames.make_one_shot_iterator()
    return filenames_iterator

def read():
    params = TrainParams()
        
    tf.reset_default_graph()
    sess = tf.InteractiveSession()

#     filenames = tf.data.Dataset.list_files(params.train_path).take(params.total_train_sample)
#     filenames_iterator = filenames.make_one_shot_iterator()
#     next_filenames = filenames_iterator.get_next()
    
    files = tf.data.TFRecordDataset('/Users/paul/Work/ai/images/tf/0main.tfr')
    files = files.map(lambda x: process_tf(x, params.input_shape[0:2]))
    files = files.batch(params.batch_size)
    files_iterator = files.make_initializable_iterator()
    next_files = files_iterator.get_next()
    sess.run(files_iterator.initializer)
    while True:
        try:
            batch = sess.run(next_files)
            # for img in batch:
            print(batch.shape)
        except tf.errors.OutOfRangeError:
            break

    print("Done")
    
read()